Irfans code. 
- Just for this phase I had to combine the code to upload it in canvas. 
- We will clean up the increment 3 code and remove any redundancy where needed. 

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df = pd.read_csv('/content/netflix_titles.csv') #read data
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...


In [ ]:
df['description'] = df['description'].str.lower()
df['title'] = df['title'].str.lower()

def tokenization(token_colum):
  column_to_tokenize = token_colum['description']
  token = nltk.word_tokenize(column_to_tokenize)
  words = [w for w in token if w.isalpha()]
  return words

df['description_tokens'] = df.apply(tokenization,axis=1)
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,description_tokens
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",in a future where the elite inhabit an island ...,"[in, a, future, where, the, elite, inhabit, an..."


In [ ]:
sw = set (stopwords.words('english'))

def stpWords_fun(stpWords):
  lst_stpw = stpWords['description_tokens']
  cleaned = [w for w in lst_stpw if not w in sw]
  return cleaned

df['stopwords_removed'] = df.apply(stpWords_fun,axis=1)
df.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,description_tokens,stopwords_removed
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",in a future where the elite inhabit an island ...,"[in, a, future, where, the, elite, inhabit, an...","[future, elite, inhabit, island, paradise, far..."


In [ ]:
lemmatizer = WordNetLemmatizer()

def lem_fun(column):
  to_lem = column['stopwords_removed']
  lemmed = [lemmatizer.lemmatize(w) for w in to_lem]
  return lemmed

df['description_lem'] = df.apply(lem_fun, axis=1)
df['description_lem']

0       [future, elite, inhabit, island, paradise, far...
1       [devastating, earthquake, hit, mexico, city, t...
2       [army, recruit, found, dead, fellow, soldier, ...
3       [postapocalyptic, world, robot, hide, fear, da...
4       [brilliant, group, student, become, expert, in...
                              ...                        
7782    [lebanon, civil, war, deprives, zozo, family, ...
7783    [scrappy, poor, boy, worm, way, tycoon, dysfun...
7784    [documentary, south, african, rapper, nasty, c...
7785    [dessert, wizard, adriano, zumbo, look, next, ...
7786    [documentary, delf, mystique, behind, trio, ex...
Name: description_lem, Length: 7787, dtype: object

In [ ]:
def rejoin_words(row):
    my_list = row['description_lem']
    joined_words = ( " ".join(my_list))
    return joined_words

df['description_lem_join'] = df.apply(rejoin_words, axis=1)
df['description_lem_join']

0       future elite inhabit island paradise far crowd...
1       devastating earthquake hit mexico city trapped...
2       army recruit found dead fellow soldier forced ...
3       postapocalyptic world robot hide fear dangerou...
4       brilliant group student become expert intent s...
                              ...                        
7782    lebanon civil war deprives zozo family left gr...
7783    scrappy poor boy worm way tycoon dysfunctional...
7784    documentary south african rapper nasty c hit s...
7785    dessert wizard adriano zumbo look next willy w...
7786    documentary delf mystique behind trio explores...
Name: description_lem_join, Length: 7787, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(df['description_lem_join'])
print(vectorizer.get_feature_names())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from sklearn.metrics.pairwise import linear_kernel

linear_model = linear_kernel(X,X)

In [ ]:
title_index = pd.Series(df.index, index=df['title']).drop_duplicates()

In [ ]:
def get_recommendations(title, linear_model=linear_model):
  title = title.lower()
  index = title_index[title]
  scores = list(enumerate(linear_model[index]))
  scores = sorted(scores, key=lambda x: x[1], reverse=True)
  scores = scores[1:11]
  movie_index = [i[0] for i in scores]

  
  return df['title'].iloc[movie_index],scores
    


In [ ]:
list (df.title)

In [ ]:
get_recommendations('black butler')

(4821                                       penny dreadful
 3566                              lee daniels' the butler
 7055        thomas & friends: thomas and the royal engine
 6739                                            the paper
 3371                              kidnapping mr. heineken
 3039    inuyasha the movie: affections touching across...
 6253                                             the crow
 4972                          prince jai aur dumdaar viru
 2422                              girl on the third floor
 7689                              xxx: state of the union
 Name: title, dtype: object,
 [(4821, 0.06461914035627946),
  (3566, 0.029710471428919156),
  (7055, 0.029106848110973575),
  (6739, 0.028795079823257616),
  (3371, 0.02788417564727782),
  (3039, 0.027812268177818896),
  (6253, 0.027118248570437212),
  (4972, 0.025762588802431815),
  (2422, 0.02567924450696608),
  (7689, 0.025091098812135573)])

In [ ]:
#splitting dataframe in a particular size 
df2 = df.sample(frac=0.6,random_state=200) 
df2.reset_index()


In [ ]:
df2.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'description_tokens', 'stopwords_removed', 'description_lem',
       'description_lem_join'],
      dtype='object')

In [ ]:
df2 = df.drop(columns=['show_id','country', 'date_added','release_year','duration', 'description','description_tokens', 'stopwords_removed','description_lem'])

In [ ]:
df.to_csv('processed_data.csv', index=False)

In [ ]:
df2.head(1)

,type,title,director,cast,rating,listed_in,description_lem_join
0,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &...",future elite inhabit island paradise far crowd...


In [ ]:
df2.isnull().sum()

type                       0
title                      0
director                2389
cast                     718
rating                     7
listed_in                  0
description_lem_join       0
dtype: int64

In [ ]:
df2=df2.fillna('')
df2

,type,title,director,cast,rating,listed_in,description_lem_join
0,TV Show,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &...",future elite inhabit island paradise far crowd...
1,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",TV-MA,"Dramas, International Movies",devastating earthquake hit mexico city trapped...
2,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",R,"Horror Movies, International Movies",army recruit found dead fellow soldier forced ...
3,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",PG-13,"Action & Adventure, Independent Movies, Sci-Fi...",postapocalyptic world robot hide fear dangerou...
4,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",PG-13,Dramas,brilliant group student become expert intent s...
...,...,...,...,...,...,...,...
7782,Movie,zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...",TV-MA,"Dramas, International Movies",lebanon civil war deprives zozo family left gr...
7783,Movie,zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",TV-14,"Dramas, International Movies, Music & Musicals",scrappy poor boy worm way tycoon dysfunctional...
7784,Movie,zulu man in japan,,Nasty C,TV-MA,"Documentaries, International Movies, Music & M...",documentary south african rapper nasty c hit s...
7785,TV Show,zumbo's just desserts,,"Adriano Zumbo, Rachel Khoo",TV-PG,"International TV Shows, Reality TV",dessert wizard adriano zumbo look next willy w...


In [ ]:
df2['type'] = pd.factorize(df2.type)[0] #factorize category 
df2['rating'] = pd.factorize(df2.rating)[0] #factorize category 
df2['title_id'] = pd.factorize(df2.title)[0] #factorize category 
df2['listed_in'] = pd.factorize(df2.title)[0] #factorize category 
df2['cast'] = pd.factorize(df2.title)[0] #factorize category 



In [ ]:
df2.head(10)

,type,title,director,cast,rating,listed_in,description_lem_join,title_id
0,0,3%,,0,0,0,future elite inhabit island paradise far crowd...,0
1,1,7:19,Jorge Michel Grau,1,0,1,devastating earthquake hit mexico city trapped...,1
2,1,23:59,Gilbert Chan,2,1,2,army recruit found dead fellow soldier forced ...,2
3,1,9,Shane Acker,3,2,3,postapocalyptic world robot hide fear dangerou...,3
4,1,21,Robert Luketic,4,2,4,brilliant group student become expert intent s...,4
5,0,46,Serdar Akar,5,0,5,genetics professor experiment treatment comato...,5
6,1,122,Yasir Al Yasiri,6,0,6,awful accident couple admitted grisly hospital...,6
7,1,187,Kevin Reynolds,7,1,7,one high school student attack dedicated teach...,7
8,1,706,Shravan Kumar,8,3,8,doctor go missing psychiatrist wife treat biza...,8
9,1,1920,Vikram Bhatt,9,0,9,architect wife move castle slated become luxur...,9


In [ ]:
df2.to_csv('processed_data.csv', index=False)

Divyanshi Code 

In [ ]:
#now showing the top 5 movies similar to TMNT accroding to this algorithm
i = 0
for item in sorted_score:
    movie_title = df[df['id'] == item[0]]['title'].values[0]
    print(i+1,movie_title)
    i = i+1
    if i > 4:
        break

1 Teenage Mutant Ninja Turtles II: The Secret of the Ooze
2 Teenage Cocktail
3 Ninja Hattori
4 Power Rangers Ninja Steel
5 Power Rangers Mystic Force


In [ ]:
# Using TF-IDF Technique to create recommendation model with Sigmoid Kernal for movies and TV shows. The result helps in finding out similar movies and TV shows with the accuracy. 


# Import numpy and pandas

import numpy as np 
import pandas as pd

# Read the dataset

data = pd.read_csv("netflix_titles.csv")
print (data.shape)
data.head(5)

(7787, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [ ]:
# Print the data columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [ ]:
# Dropping the columns thata are not required

new_movies = data[data['type']=='Movie'].reset_index()
new_movies = new_movies.drop(columns = ['duration','country','date_added','release_year','show_id','type','index','listed_in'])

In [ ]:
 # Searching if there is any null values

new_movies['director'] = new_movies['director'].fillna("")
new_movies['cast'] = new_movies['cast'].fillna("")

# Combining columns such as description, cast and directors

new_movies['combined'] = new_movies['description']+new_movies['cast']+movies['director']
new_movies.head(5)

,title,director,cast,rating,description,combined
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",TV-MA,After a devastating earthquake hits Mexico Cit...,After a devastating earthquake hits Mexico Cit...
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",R,"When an army recruit is found dead, his fellow...","When an army recruit is found dead, his fellow..."
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",PG-13,"In a postapocalyptic world, rag-doll robots hi...","In a postapocalyptic world, rag-doll robots hi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",PG-13,A brilliant group of students become card-coun...,A brilliant group of students become card-coun...
4,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",TV-MA,"After an awful accident, a couple admitted to ...","After an awful accident, a couple admitted to ..."


In [ ]:
# Importing TfidVectorizer to find out the best recommendation

from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3,max_features = None,analyzer = 'word',token_pattern = 'r\w{1,}', ngram_range = (1,3), stop_words = 'english')

In [ ]:
movies['combined'] = movies['combined'].fillna("")

In [ ]:
# Using Sigmoid Kernal and creating the matrix

tfv_matrix = tfv.fit_transform(new_movies['combined'])
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfv_matrix,tfv_matrix)
sig[0]
indices = pd.Series(new_movies.index,index = new_movies['title']).drop_duplicates()
indices

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ral', 'rd', 'rds', 'ready', 'reafter', 'reas', 'reby', 'red', 'ree', 'refore', 'rehand', 'rein', 'reover', 'rest', 'reupon', 'rever', 'rhaps', 'ribe', 'ring', 'rious', 'rly', 'rmer', 'rmerly', 'rom', 'ront', 'ross', 'rough', 'roughout', 'round', 'rs', 'rself', 'rselves', 'rst', 'rt', 'rtheless', 'rther', 'rty', 'ru', 'rwards', 'rwise', 'ry', 'ryone', 'rything', 'rywhere'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


title
7:19                                          0
23:59                                         1
9                                             2
21                                            3
122                                           4
                                           ... 
Zoom                                       5372
Zozo                                       5373
Zubaan                                     5374
Zulu Man in Japan                          5375
ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS    5376
Length: 5377, dtype: int64

In [ ]:
# Creating a recommendation model to find similar movies 

def recommend(title,sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    movies_indices = [i[0] for i in sig_scores]
    return new_movies['title'].iloc[movies_indices],sig_scores
  

In [ ]:
recommend('Avengers: Infinity War')

(4895               Thor: Ragnarok
 3587     Resident Evil: Afterlife
 4140         Take Me Home Tonight
 1878                          Her
 4707     The Pursuit of Happyness
 605                 Before I Fall
 975     Chris D'Elia: Man on Fire
 179                 A Secret Love
 3449           Prescription Thugs
 5349         Yours, Mine and Ours
 Name: title, dtype: object,
 [(4895, 0.7616270181011502),
  (3587, 0.7616232966831555),
  (4140, 0.7616193098679381),
  (1878, 0.761617602870682),
  (4707, 0.7616143417188269),
  (605, 0.7616131942359952),
  (975, 0.7616130803234207),
  (179, 0.7616128661965993),
  (3449, 0.7616128656226869),
  (5349, 0.76161253836603)])

In [ ]:
recommend('Tarzan')

(4158                               Tarif de nuit
 4334                                 The Command
 808     Brian Regan: Nunchucks and Flamethrowers
 4707                    The Pursuit of Happyness
 2788                                     Maynard
 2082          Indiana Jones and the Last Crusade
 4171                                Teach Us All
 2182                 Jeff Dunham: Beside Himself
 3700                            S Is for Stanley
 4898                          Thorne: Sleepyhead
 Name: title, dtype: object,
 [(4158, 0.7616181393619372),
  (4334, 0.7616172582464316),
  (808, 0.7616150096251881),
  (4707, 0.7616148951884403),
  (2788, 0.7616146430974647),
  (2082, 0.7616145351898341),
  (4171, 0.7616138606485937),
  (2182, 0.7616135532341477),
  (3700, 0.7616131948927722),
  (4898, 0.761612689658796)])

In [ ]:
# Dropping the columns that are not required

tv_shows = data[data['type']=='TV Show'].reset_index()
tv_shows = tv_shows.drop(columns = ['duration','country','date_added','release_year','show_id','type','index','listed_in'])

In [ ]:
 # Searching if there is any null values

tv_shows['director'] = tv_shows['director'].fillna("")
tv_shows['cast'] = tv_shows['cast'].fillna("")

# Combining columns such as description, cast and directors

tv_shows['combined'] = tv_shows['description']+tv_shows['cast']+tv_shows['director']

In [ ]:
tv_shows.head()

,title,director,cast,rating,description,combined
0,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",TV-MA,In a future where the elite inhabit an island ...,In a future where the elite inhabit an island ...
1,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",TV-MA,A genetics professor experiments with a treatm...,A genetics professor experiments with a treatm...
2,1983,,"Robert Więckiewicz, Maciej Musiał, Michalina O...",TV-MA,"In this dark alt-history thriller, a naïve law...","In this dark alt-history thriller, a naïve law..."
3,1994,Diego Enrique Osorno,,TV-MA,Archival video and new interviews examine Mexi...,Archival video and new interviews examine Mexi...
4,Feb-09,,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",TV-14,"As a psychology professor faces Alzheimer's, h...","As a psychology professor faces Alzheimer's, h..."


In [ ]:
# Importing TfidVectorizer to find out the best recommendation

from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df = 3,max_features = None,analyzer = 'word',token_pattern = 'r\w{1,}', ngram_range = (1,3), stop_words = 'english')
tv_shows['combined'] = tv_shows['combined'].fillna("")

In [ ]:
# Using Sigmoid Kernal and creating the matrix

tfv_matrix_shows = tfv.fit_transform(tv_shows['combined'])

from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfv_matrix_shows,tfv_matrix_shows)
sig[0]
indices = pd.Series(tv_shows.index,index = tv_shows['title']).drop_duplicates()
indices

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ral', 'rd', 'rds', 'ready', 'reafter', 'reas', 'reby', 'red', 'ree', 'refore', 'rehand', 'rein', 'reover', 'rest', 'reupon', 'rever', 'rhaps', 'ribe', 'ring', 'rious', 'rly', 'rmer', 'rmerly', 'rom', 'ront', 'ross', 'rough', 'roughout', 'round', 'rs', 'rself', 'rselves', 'rst', 'rt', 'rtheless', 'rther', 'rty', 'ru', 'rwards', 'rwise', 'ry', 'ryone', 'rything', 'rywhere'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


title
3%                          0
46                          1
1983                        2
1994                        3
Feb-09                      4
                         ... 
Zindagi Gulzar Hai       2405
Zoids Wild               2406
Zombie Dumb              2407
Zona Rosa                2408
Zumbo's Just Desserts    2409
Length: 2410, dtype: int64

In [ ]:
# Creating a recommendation model to find similar TV Shows
 
def recommend(title,sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    shows_indices = [i[0] for i in sig_scores]
    return tv_shows['title'].iloc[shows_indices], sig_scores

In [ ]:
recommend('Iron Man: Armored Adventures')

(1977                                           The Hollow
 1923                                    The Dragon Prince
 383                       Cinderella and the Four Knights
 1076                  LEGO Jurassic World: Secret Exhibit
 2109                                        The Staircase
 1064    Learning Songs by Little Baby Bum: Nursery Rhy...
 1327             Mystery Science Theater 3000: The Return
 1159                                                Lupin
 1096               Little Baby Bum: Nursery Rhyme Friends
 759                                           Grand Hotel
 Name: title, dtype: object,
 [(1977, 0.761673673909367),
  (1923, 0.7616693914166767),
  (383, 0.7616568124636461),
  (1076, 0.7616491537416048),
  (2109, 0.7616452708075617),
  (1064, 0.7616438428587147),
  (1327, 0.7616435904246859),
  (1159, 0.7616431371729565),
  (1096, 0.7616400920896784),
  (759, 0.7616395289981868)])

In [ ]:
recommend("Grey's Anatomy")

(186                           Battle Creek
 531                Do Do Sol Sol La La Sol
 33                      A Boy Name Flora A
 385                             Cinta Iris
 386               Cinta Si Wedding Planner
 1905                             The Crime
 1650                           Secret City
 1514                             President
 859                               Hormones
 1998    The Irregular at Magic High School
 Name: title, dtype: object,
 [(186, 0.7616726260963065),
  (531, 0.7616689331625517),
  (33, 0.7616669835173516),
  (385, 0.7616617394345969),
  (386, 0.7616576078500836),
  (1905, 0.761651856791308),
  (1650, 0.7616503959954495),
  (1514, 0.7616485062865156),
  (859, 0.7616475372370397),
  (1998, 0.761647517158233)])

Srikanth code

In [ ]:
df = pd.read_csv("/Users/Administrator/PycharmProjects/pythonProject2/PytonProject/netflix_titles.csv")
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [ ]:
#create a new feature that we will use to calculate the similarity between movies, it's a combination of some other features
df["overall_infos"]= df["type"] + " "+df["title"]+" "+df["description"]

In [ ]:
#adding new column because there are mistakes in show_id column
df.insert(1, "id", list(range(1, len(df)+1)), True) 

In [ ]:
df.head()

,show_id,id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,overall_infos
0,s1,1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,TV Show 3% In a future where the elite inhabit...
1,s2,2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,Movie 7:19 After a devastating earthquake hits...
2,s3,3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",Movie 23:59 When an army recruit is found dead...
3,s4,4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...","Movie 9 In a postapocalyptic world, rag-doll r..."
4,s5,5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,Movie 21 A brilliant group of students become ...


In [ ]:
#
df_processed = df[['id','overall_infos']]
df_processed.head(1)

,id,overall_infos
0,1,TV Show 3% In a future where the elite inhabit...


In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
#processing the overall info column
def text_preprocessing(column):
    #make all words with lower letters
    column = column.str.lower()
    #getting rid of any punctution
    column = column.str.replace('http\S+|www.\S+|@|%|:|,|', '', case=False)
    #spliting each sentence to words to apply previous funtions on them 
    word_tokens = column.str.split()
    keywords = word_tokens.apply(lambda x: [item for item in x if item not in stop])
    #assemble words of each sentence again and assign them in new column
    for i in range(len(keywords)):
        keywords[i] = " ".join(keywords[i])
        column = keywords

    return column

In [ ]:
df_processed['processed_infos'] = text_preprocessing(df_processed['overall_infos'])

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_processed.head(1)

,id,overall_infos,processed_infos
0,1,TV Show 3% In a future where the elite inhabit...,tv show 3 future elite inhabit island paradise...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CV = CountVectorizer()
converted_metrix = CV.fit_transform(df_processed['processed_infos'])

In [ ]:
cosine_similarity = cosine_similarity(converted_metrix)

In [ ]:
cosine_similarity

array([[1.        , 0.        , 0.05892557, ..., 0.        , 0.10660036,
        0.        ],
       [0.        , 1.        , 0.0571662 , ..., 0.17149859, 0.        ,
        0.05057217],
       [0.05892557, 0.0571662 , 1.        , ..., 0.05555556, 0.        ,
        0.04914732],
       ...,
       [0.        , 0.17149859, 0.05555556, ..., 1.        , 0.        ,
        0.09829464],
       [0.10660036, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.05057217, 0.04914732, ..., 0.09829464, 0.        ,
        1.        ]])

In [ ]:
#recommendation based on title, type, description
def get_recommendations(title):    
    if title is not None:
        recom = df[df['overall_infos'].str.contains(title)]
    return recom

In [ ]:
title = input("Enter title name: ")
get_recommendations(title)

Enter title name: comedy


,show_id,id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,overall_infos
29,s30,30,TV Show,#blackAF,NaN,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,"April 17, 2020",2020,TV-MA,1 Season,TV Comedies,Kenya Barris and his family navigate relations...,TV Show #blackAF Kenya Barris and his family n...
136,s137,137,Movie,7 Khoon Maaf,Vishal Bhardwaj,"Priyanka Chopra, Neil Nitin Mukesh, John Abrah...",India,"August 2, 2018",2011,TV-MA,148 min,"Dramas, International Movies, Thrillers","Spiced liberally with black comedy, this Bolly...",Movie 7 Khoon Maaf Spiced liberally with black...
174,s175,175,Movie,A Futile and Stupid Gesture,David Wain,"Will Forte, Domhnall Gleeson, Martin Mull, Joe...",United States,"January 26, 2018",2018,TV-MA,102 min,Comedies,"In a brief life full of triumph and failure, ""...",Movie A Futile and Stupid Gesture In a brief l...
210,s211,211,Movie,A Night at the Roxbury,John Fortenberry,"Will Ferrell, Chris Kattan, Dan Hedaya, Molly ...",United States,"December 1, 2019",1998,PG-13,82 min,"Comedies, Cult Movies","After a run-in with Richard Grieco, dimwits Do...",Movie A Night at the Roxbury After a run-in wi...
307,s308,308,Movie,ADAM SANDLER 100% FRESH,Steve Brill,Adam Sandler,United States,"October 23, 2018",2018,TV-MA,74 min,Stand-Up Comedy,"From ""Heroes"" to ""Ice Cream Ladies"" – Adam San...","Movie ADAM SANDLER 100% FRESH From ""Heroes"" to..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7271,s7272,7272,TV Show,Turbo FAST,NaN,"Reid Scott, John Eric Bentley, Amir Talai, Phi...",United States,"February 5, 2016",2016,TV-Y7,3 Seasons,Kids' TV,"Join Turbo and his posse, the Fast Action Stun...","TV Show Turbo FAST Join Turbo and his posse, t..."
7450,s7451,7451,TV Show,W/ Bob & David,NaN,"David Cross, Bob Odenkirk, John Ennis, Jay Joh...",United States,"November 13, 2015",2015,TV-MA,1 Season,TV Comedies,After being dishonorably discharged from the N...,TV Show W/ Bob & David After being dishonorabl...
7477,s7478,7478,Movie,War Chhod Na Yaar,Faraz Haider,"Sharman Joshi, Soha Ali Khan, Javed Jaffrey, D...",India,"March 2, 2019",2013,TV-14,109 min,"Comedies, Dramas, International Movies","Set in an India-Pakistan border post, this sat...",Movie War Chhod Na Yaar Set in an India-Pakist...
7565,s7566,7566,Movie,What the Jatt!!,Saket Behl,"Harish Verma, Isha Rikhi, Vipul Roy, Binnu Dhi...",India,"October 1, 2018",2015,TV-PG,140 min,"Comedies, International Movies, Romantic Movies",A peasant from Punjab and his best friend pose...,Movie What the Jatt!! A peasant from Punjab an...


In [ ]:
#this how we will get the id of the movie so we can check similarity between it and other movies
title = 'Teenage Mutant Ninja Turtles'

movie_id = df[df['title'] == title]['id'].values[0]

In [ ]:
#simil
score = list(enumerate(cosine_similarity[movie_id]))

In [ ]:
#now sort the similar movies in descending order
sorted_score = sorted(score, key=lambda x:x[1], reverse= True)
#we will ignore the first score because it will give us a 100% score because it's the same movie 
sorted_score = sorted_score[1:]

In [ ]:
#now showing the top 5 movies similar to TMNT accroding to this algorithm
i = 0
for item in sorted_score:
    movie_title = df[df['id'] == item[0]]['title'].values[0]
    print(i+1,movie_title)
    i = i+1
    if i > 4:
        break

1 Teenage Mutant Ninja Turtles II: The Secret of the Ooze
2 Teenage Cocktail
3 Ninja Hattori
4 Power Rangers Ninja Steel
5 Power Rangers Mystic Force
